In [2]:
import requests
import datetime
import os
from typing import Optional
import pandas as pd
import glob
import os

os.chdir("/Users/andrewfish/Developer/Financial Data")

def download_opra_files(dates):
    """
    Consolidates the download logic into one function that downloads
    three types of gzipped CSV files (trades, minute_aggs, day_aggs)
    for each date in the provided list of dates.
    Returns a list of all successfully downloaded file paths.
    """
    def download_b2_file(file_path: str) -> Optional[str]:
        """
        Helper function describing how the file will be downloaded
        from Backblaze using the new endpoint with a destination_dir.
        """
        endpoint = "http://localhost:8000/backblaze/download"
        
        # Determine subdirectory based on file path
        if "trades_v1" in file_path:
            sub_dir = "trades"
        elif "minute_aggs_v1" in file_path:
            sub_dir = "minute_aggs"
        elif "day_aggs_v1" in file_path:
            sub_dir = "day_aggs"
        else:
            # If no recognized pattern, defaults to data/downloads
            sub_dir = ""

        destination_dir = os.path.join("data", "downloads", sub_dir)
        expected_path = os.path.join(destination_dir, os.path.basename(file_path))

        try:
            # Pass file_path and destination_dir as query params to the endpoint
            response = requests.post(
                endpoint, 
                params={
                    "file_path": file_path,
                    "destination_dir": destination_dir
                }
            )
            response.raise_for_status()  # Raise exception for HTTP errors
            return expected_path
        except requests.exceptions.RequestException as e:
            #print(f"Failed to download {file_path} - error: {e}")
            return None

    downloaded_files = []
    failed_files = []

    # Collect all file paths for each date
    for date in dates:
        paths_to_download = [
            f"us_options_opra/trades_v1/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%Y-%m-%d')}.csv.gz",
            f"us_options_opra/minute_aggs_v1/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%Y-%m-%d')}.csv.gz",
            f"us_options_opra/day_aggs_v1/{date.strftime('%Y')}/{date.strftime('%m')}/{date.strftime('%Y-%m-%d')}.csv.gz"
        ]

        for file_path in paths_to_download:
            result = download_b2_file(file_path)
            if result:
                downloaded_files.append(result)
            else:
                failed_files.append(file_path)

    # Report summary
    print("\nDownload Summary:")
    print(f"Successfully downloaded: {len(downloaded_files)} files")
    print(f"Failed downloads: {len(failed_files)} files")

    return downloaded_files

In [9]:
# Create a list of dates to download
start_date = datetime.datetime(2024, 6, 30)
end_date = datetime.datetime(2024, 12, 24)

dates_to_download = []
current_date = start_date
while current_date <= end_date:
    dates_to_download.append(current_date)
    current_date += datetime.timedelta(days=1)

# Download the files
downloaded_files = download_opra_files(dates_to_download)


Download Summary:
Successfully downloaded: 372 files
Failed downloads: 162 files


In [3]:
# Get lists of files for each type
#trade_files = glob.glob('data/downloads/trades/*.csv.gz')
#minute_agg_files = glob.glob('data/downloads/minute_aggs/*.csv.gz')
day_agg_files = glob.glob('data/downloads/day_aggs/*.csv.gz')

# Create DataFrames for each type
#trades_df = pd.concat([pd.read_csv(f) for f in trade_files], ignore_index=True)
#minute_aggs_df = pd.concat([pd.read_csv(f) for f in minute_agg_files], ignore_index=True)
day_aggs_df = pd.concat([pd.read_csv(f) for f in day_agg_files], ignore_index=True)

# Print basic information about each DataFrame
#print("\nTrades DataFrame:")
#print(trades_df.info())
#print("\nMinute Aggregations DataFrame:")
#print(minute_aggs_df.info())
print("\nDaily Aggregations DataFrame:")
print(day_aggs_df.info())


Daily Aggregations DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28146631 entries, 0 to 28146630
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   ticker        object 
 1   volume        int64  
 2   open          float64
 3   close         float64
 4   high          float64
 5   low           float64
 6   window_start  int64  
 7   transactions  int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 1.7+ GB
None


In [4]:
day_aggs_df.head()

,ticker,volume,open,close,high,low,window_start,transactions
0,O:A241220C00110000,25,31.65,31.70,31.70,31.65,1733893200000000000,3
1,O:A241220C00130000,833,12.45,12.00,12.86,11.90,1733893200000000000,81
2,O:A241220C00135000,17,7.70,7.59,7.70,7.59,1733893200000000000,7
3,O:A241220C00140000,19,3.70,3.40,4.20,3.40,1733893200000000000,7
4,O:A241220C00145000,16,1.80,1.10,1.80,1.10,1733893200000000000,8


In [13]:
# Fix the underlying_ticker extraction to only get the actual ticker part
day_aggs_df['underlying_ticker'] = day_aggs_df['ticker'].str.extract(r'O:([A-Z]+\d*?)(?:\d{6}[CP])')[0]

# Display the result
print(day_aggs_df.head())

               ticker  volume   open  close   high    low  \
0  O:A241220C00110000      25  31.65  31.70  31.70  31.65   
1  O:A241220C00130000     833  12.45  12.00  12.86  11.90   
2  O:A241220C00135000      17   7.70   7.59   7.70   7.59   
3  O:A241220C00140000      19   3.70   3.40   4.20   3.40   
4  O:A241220C00145000      16   1.80   1.10   1.80   1.10   

          window_start  transactions underlying_ticker expiration_date  
0  1733893200000000000             3                 A      2024-12-20  
1  1733893200000000000            81                 A      2024-12-20  
2  1733893200000000000             7                 A      2024-12-20  
3  1733893200000000000             7                 A      2024-12-20  
4  1733893200000000000             8                 A      2024-12-20  


In [10]:
# Extract expiration date from the option symbols and convert to datetime
day_aggs_df['expiration_date'] = (
    day_aggs_df['ticker']
    .str.extract(r'[A-Z]+\d*(\d{6})[CP]')[0]  # Modified pattern to handle numbers in ticker
    .apply(lambda x: datetime.datetime.strptime('20' + x, '%Y%m%d'))
)

# Display the result to verify
print(day_aggs_df.head())

               ticker underlying_ticker expiration_date
0  O:A241220C00110000                 A      2024-12-20
1  O:A241220C00130000                 A      2024-12-20
2  O:A241220C00135000                 A      2024-12-20
3  O:A241220C00140000                 A      2024-12-20
4  O:A241220C00145000                 A      2024-12-20

Updated underlying tickers:
               ticker underlying_ticker
0  O:A241220C00110000           A241220
1  O:A241220C00130000           A241220
2  O:A241220C00135000           A241220
3  O:A241220C00140000           A241220
4  O:A241220C00145000           A241220


In [14]:
# Extract contract type (P/C) from the option symbols
day_aggs_df['type'] = day_aggs_df['ticker'].str.extract(r'(\d{6})([CP])')[1]

# Display the result
print(day_aggs_df.head())

               ticker  volume   open  close   high    low  \
0  O:A241220C00110000      25  31.65  31.70  31.70  31.65   
1  O:A241220C00130000     833  12.45  12.00  12.86  11.90   
2  O:A241220C00135000      17   7.70   7.59   7.70   7.59   
3  O:A241220C00140000      19   3.70   3.40   4.20   3.40   
4  O:A241220C00145000      16   1.80   1.10   1.80   1.10   

          window_start  transactions underlying_ticker expiration_date type  
0  1733893200000000000             3                 A      2024-12-20    C  
1  1733893200000000000            81                 A      2024-12-20    C  
2  1733893200000000000             7                 A      2024-12-20    C  
3  1733893200000000000             7                 A      2024-12-20    C  
4  1733893200000000000             8                 A      2024-12-20    C  


In [15]:
# Extract strike price from the option symbols
day_aggs_df['strike'] = (
    day_aggs_df['ticker']
    .str.extract(r'[CP](\d+)$')[0]  # Extract all digits after P or C until the end
    .astype(float) / 1000  # Convert to float and divide by 1000
)

# Display the result
print(day_aggs_df.head())

               ticker  volume   open  close   high    low  \
0  O:A241220C00110000      25  31.65  31.70  31.70  31.65   
1  O:A241220C00130000     833  12.45  12.00  12.86  11.90   
2  O:A241220C00135000      17   7.70   7.59   7.70   7.59   
3  O:A241220C00140000      19   3.70   3.40   4.20   3.40   
4  O:A241220C00145000      16   1.80   1.10   1.80   1.10   

          window_start  transactions underlying_ticker expiration_date type  \
0  1733893200000000000             3                 A      2024-12-20    C   
1  1733893200000000000            81                 A      2024-12-20    C   
2  1733893200000000000             7                 A      2024-12-20    C   
3  1733893200000000000             7                 A      2024-12-20    C   
4  1733893200000000000             8                 A      2024-12-20    C   

   strike  
0   110.0  
1   130.0  
2   135.0  
3   140.0  
4   145.0  


In [16]:
# Create the directory if it doesn't exist
os.makedirs('data/processed/day_aggs', exist_ok=True)

# Save the processed DataFrame
day_aggs_df.to_csv('data/processed/day_aggs/processed_day_aggs.csv', index=False)

print("File saved successfully to data/processed/day_aggs/processed_day_aggs.csv")

File saved successfully to data/processed/day_aggs/processed_day_aggs.csv


In [19]:
# Calculate the number of unique values for 'underlying_ticker'
unique_underlying_tickers = day_aggs_df['underlying_ticker'].nunique()

# Get the top tickers by volume
top_tickers_by_volume = (
    day_aggs_df.groupby('underlying_ticker')['volume']
    .sum()
    .sort_values(ascending=False)
    .head(20)  # Show top 20 for now
)

# Get the top tickers by number of options
top_tickers_by_options = (
    day_aggs_df.groupby('underlying_ticker').size()
    .sort_values(ascending=False)
    .head(20)  # Show top 20 for now
)

print(f"Number of unique underlying tickers: {unique_underlying_tickers}")
print("\nTop 20 tickers by total volume:")
print(top_tickers_by_volume)
print("\nTop 20 tickers by number of options:")
print(top_tickers_by_options)

Number of unique underlying tickers: 6057

Top 20 tickers by total volume:
underlying_ticker
SPY     939036578
QQQ     488488864
NVDA    482102884
SPXW    302572827
TSLA    267744644
IWM     190334838
VIX     107789493
AAPL    105612376
AMD      73154011
AMZN     70015318
PLTR     66198497
SPX      63333882
TLT      61295329
SMCI     54783477
INTC     48330310
META     42997284
SLV      42460555
FXI      39313660
HYG      38670918
MSTR     37861203
Name: volume, dtype: int64

Top 20 tickers by number of options:
underlying_ticker
SPXW    747081
SPY     570526
NVDA    440226
QQQ     408337
TSLA    285679
SPX     266940
IWM     251744
MSTR    231374
NDXP    217809
META    210019
SMCI    203790
XSP     181165
MSFT    158105
AVGO    157600
NFLX    144024
COIN    140986
TLT     140915
AMD     139982
RUTW    139154
AAPL    132215
dtype: int64


In [21]:
# Common ETF and index prefixes/tickers to exclude
etf_patterns = [
    '^VIX', '^SPX', '^SPY', '^QQQ', '^IWM', '^DIA',  # Common indices
    'VXX', 'UVXY', 'TVIX',  # Volatility ETFs
    'SPY', 'QQQ', 'IWM', 'EEM', 'XLF', 'EFA', 'HYG', 'TLT',  # Popular ETFs
    'VTI', 'VOO', 'VEA', 'BND', 'VWO', 'VUG', 'VTV',  # Vanguard ETFs
    'DIA', 'XLE', 'XLK', 'XLV', 'XLI', 'XLP', 'XLU',  # Sector ETFs
    'SLV', 'GLD', 'USO', 'UNG',  # Commodity ETFs
     # Indices
    'SPXW', 'SPX', 'VIX', 'NDX', 'RUT',
    
    # Leveraged ETFs
    'TQQQ', 'SQQQ', 'UVXY', 'SOXL', 'SPXU', 'SPXS',
    
    # Country/Region ETFs
    'FXI', 'EEM', 'EWZ', 'EFA', 'EWJ',
    
    # Sector/Asset ETFs
    'XLE', 'XLF', 'XLK', 'XLI', 'XLP', 'GLD', 'SLV', 'USO',
    
    # Broad Market ETFs
    'SPY', 'QQQ', 'IWM', 'DIA', 'VTI', 'VOO'
]

# Create a mask for non-ETF tickers
non_etf_mask = ~day_aggs_df['underlying_ticker'].isin(etf_patterns)

# Filter the DataFrame
filtered_df = day_aggs_df[non_etf_mask].copy()

# Show the comparison
print("Original number of unique tickers:", day_aggs_df['underlying_ticker'].nunique())
print("Filtered number of unique tickers:", filtered_df['underlying_ticker'].nunique())

# Show top 20 remaining tickers by volume
top_remaining = (
    filtered_df.groupby('underlying_ticker')['volume']
    .sum()
    .sort_values(ascending=False)
    .head(20)
)
print("\nTop 20 remaining tickers by volume:")
print(top_remaining)

Original number of unique tickers: 6057
Filtered number of unique tickers: 6016

Top 20 remaining tickers by volume:
underlying_ticker
NVDA     482102884
TSLA     267744644
AAPL     105612376
AMD       73154011
AMZN      70015318
PLTR      66198497
SMCI      54783477
INTC      48330310
META      42997284
MSTR      37861203
MARA      37765405
MSFT      36138289
SOFI      36085204
GOOGL     33145800
AVGO      30673879
COIN      28109235
GME       27686332
BABA      25341547
MU        25166866
BITO      24810576
Name: volume, dtype: int64


In [23]:
# Get the top 1000 tickers by total options volume
top_1000_tickers = (
    filtered_df.groupby('underlying_ticker')['volume']
    .sum()
    .sort_values(ascending=False)
    .head(1000)
    .index
    .tolist()
)

# Filter the DataFrame to only include these tickers
filtered_df = filtered_df[filtered_df['underlying_ticker'].isin(top_1000_tickers)].copy()

# Print some statistics about the filtered dataset
print(f"Number of unique tickers in filtered dataset: {filtered_df['underlying_ticker'].nunique()}")
print("\nTop 20 tickers by volume:")
print(
    filtered_df.groupby('underlying_ticker')['volume']
    .sum()
    .sort_values(ascending=False)
    .head(20)
)

Number of unique tickers in filtered dataset: 1000

Top 20 tickers by volume:
underlying_ticker
NVDA     482102884
TSLA     267744644
AAPL     105612376
AMD       73154011
AMZN      70015318
PLTR      66198497
SMCI      54783477
INTC      48330310
META      42997284
MSTR      37861203
MARA      37765405
MSFT      36138289
SOFI      36085204
GOOGL     33145800
AVGO      30673879
COIN      28109235
GME       27686332
BABA      25341547
MU        25166866
BITO      24810576
Name: volume, dtype: int64


In [25]:
# Get unique dates and tickers
all_dates = filtered_df['expiration_date'].unique()
all_tickers = filtered_df['underlying_ticker'].unique()

# Create a cross product of dates and tickers
date_index = pd.MultiIndex.from_product(
    [all_dates, all_tickers],
    names=['date', 'underlying_ticker']
)

# Create the base DataFrame with one row per date per ticker
base_df = pd.DataFrame(index=date_index).reset_index()

# Sort the DataFrame by date and ticker
base_df = base_df.sort_values(['date', 'underlying_ticker'])

# Save this base dataset
base_df.to_csv('data/processed/day_aggs/base_df.csv', index=False)

# Display some information about the new DataFrame
print(f"Total rows in base DataFrame: {len(base_df)}")
print(f"Number of unique dates: {len(all_dates)}")
print(f"Number of unique tickers: {len(all_tickers)}")
print("\nFirst few rows:")
print(base_df.head())

Total rows in base DataFrame: 183000
Number of unique dates: 183
Number of unique tickers: 1000

First few rows:
             date underlying_ticker
180000 2024-07-01                AA
180001 2024-07-01               AAL
180002 2024-07-01              AAOI
180003 2024-07-01               AAP
180004 2024-07-01              AAPL


In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

# Convert dates to datetime if they aren't already
base_df['date'] = pd.to_datetime(base_df['date'])

# Get the overall date range
start_date = base_df['date'].min()
end_date = base_df['date'].max()

# Function to fetch data for a single ticker
def fetch_ticker_data(ticker):
    try:
        # Fetch data with yfinance
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date + timedelta(days=1))
        
        # Reset index to make date a column and rename columns
        hist = hist.reset_index()
        hist.columns = hist.columns.str.lower()
        
        # Rename 'date' column if it's named differently
        if 'date' not in hist.columns:
            hist = hist.rename(columns={'index': 'date'})
        
        # Add ticker column
        hist['underlying_ticker'] = ticker
        
        return hist[['date', 'underlying_ticker', 'open', 'high', 'low', 'close', 'volume']]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

# Create empty list to store DataFrames
all_data = []

# Process tickers in batches to show progress
total_tickers = len(all_tickers)
for i, ticker in enumerate(all_tickers, 1):
    if i % 10 == 0:  # Print progress every 10 tickers
        print(f"Processing ticker {i} of {total_tickers}: {ticker}")
    
    ticker_data = fetch_ticker_data(ticker)
    if ticker_data is not None:
        all_data.append(ticker_data)

# Combine all data
if all_data:
    market_data = pd.concat(all_data, ignore_index=True)
    
    # Merge with base_df
    result_df = pd.merge(
        base_df,
        market_data,
        on=['date', 'underlying_ticker'],
        how='left'
    )
    
    # Save the result
    result_df.to_csv('data/processed/day_aggs/base_df_with_market_data.csv', index=False)
    
    # Display some information about the result
    print("\nData fetching complete!")
    print(f"Total rows in final DataFrame: {len(result_df)}")
    print("\nSample of the data:")
    print(result_df.head())
    
    # Check for any missing data
    missing_data = result_df[result_df['close'].isna()]
    if len(missing_data) > 0:
        print(f"\nNumber of rows with missing data: {len(missing_data)}")
        print("Sample of tickers with missing data:")
        print(missing_data['underlying_ticker'].value_counts().head())
else:
    print("No data was successfully fetched!")

In [27]:
# Convert market_data dates to naive datetime (remove timezone)
market_data['date'] = market_data['date'].dt.tz_localize(None)

# Now try the merge again
result_df = pd.merge(
    base_df,
    market_data,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Save the result
result_df.to_csv('data/processed/day_aggs/base_df_with_market_data.csv', index=False)

# Display some information about the result
print("\nData fetching complete!")
print(f"Total rows in final DataFrame: {len(result_df)}")
print("\nSample of the data:")
print(result_df.head())

# Check for any missing data
missing_data = result_df[result_df['close'].isna()]
if len(missing_data) > 0:
    print(f"\nNumber of rows with missing data: {len(missing_data)}")
    print("Sample of tickers with missing data:")
    print(missing_data['underlying_ticker'].value_counts().head())


Data fetching complete!
Total rows in final DataFrame: 183000

Sample of the data:
        date underlying_ticker        open        high         low  \
0 2024-07-01                AA   40.021413   40.637890   39.315446   
1 2024-07-01               AAL   11.330000   11.410000   11.000000   
2 2024-07-01              AAOI    8.390000    8.660000    7.850000   
3 2024-07-01               AAP   61.909940   62.147368   58.813453   
4 2024-07-01              AAPL  211.611974  217.019756  211.442359   

        close      volume  
0   39.474537   3228000.0  
1   11.040000  28624300.0  
2    8.080000   1778500.0  
3   59.070667   1767600.0  
4  216.261475  60402900.0  

Number of rows with missing data: 74095
Sample of tickers with missing data:
underlying_ticker
BABA1    183
WHR      183
WYNN     183
WW       183
WULF     183
Name: count, dtype: int64


In [28]:
# First, let's see what percentage of our data is missing
total_rows = len(result_df)
missing_rows = len(missing_data)
print(f"Missing data percentage: {(missing_rows/total_rows)*100:.2f}%")

# Check if missing data is concentrated on specific dates (like weekends)
missing_by_date = missing_data['date'].value_counts().sort_index()
print("\nDates with most missing data:")
print(missing_by_date.head())

# Check tickers with 100% missing data
completely_missing = missing_data.groupby('underlying_ticker').size()
completely_missing = completely_missing[completely_missing == 183]  # 183 is the number of dates per ticker
print("\nTickers with completely missing data:")
print(completely_missing)

# Remove weekends and holidays (optional)
is_trading_day = ~result_df['close'].isna().groupby(result_df['date']).mean().round()
trading_days = is_trading_day[is_trading_day].index

# Filter for only trading days
clean_df = result_df[result_df['date'].isin(trading_days)].copy()

# Save the cleaned dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

print("\nCleaned dataset summary:")
print(f"Original rows: {len(result_df)}")
print(f"Cleaned rows: {len(clean_df)}")
print(f"Missing data in cleaned dataset: {clean_df['close'].isna().sum()}")

Missing data percentage: 40.49%

Dates with most missing data:
date
2024-07-01    131
2024-07-02    131
2024-07-03    131
2024-07-05    131
2024-07-08    131
Name: count, dtype: int64

Tickers with completely missing data:
underlying_ticker
BABA1    183
BIG      183
BRKB     183
DJX      183
GPS      183
        ... 
ZIM      183
ZION     183
ZM       183
ZS       183
ZTO      183
Length: 127, dtype: int64


TypeError: ufunc 'invert' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
import time

# Get list of tickers with completely missing data
missing_tickers = completely_missing.index.tolist()

# Function to fetch data for a single ticker (with delay)
def fetch_ticker_data(ticker):
    try:
        # Add small delay between requests
        time.sleep(0.5)  # 500ms delay
        
        # Try alternative symbols for some known cases
        if ticker == 'BRKB':
            ticker = 'BRK-B'
        
        # Fetch data with yfinance
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date + timedelta(days=1))
        
        # Reset index to make date a column and rename columns
        hist = hist.reset_index()
        hist.columns = hist.columns.str.lower()
        
        # Rename 'date' column if it's named differently
        if 'date' not in hist.columns:
            hist = hist.rename(columns={'index': 'date'})
        
        # Add ticker column (use original ticker name)
        hist['underlying_ticker'] = ticker
        
        return hist[['date', 'underlying_ticker', 'open', 'high', 'low', 'close', 'volume']]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

# Create empty list to store new data
new_data = []

# Process missing tickers
total_missing = len(missing_tickers)
for i, ticker in enumerate(missing_tickers, 1):
    print(f"Processing missing ticker {i} of {total_missing}: {ticker}")
    ticker_data = fetch_ticker_data(ticker)
    if ticker_data is not None:
        new_data.append(ticker_data)

# If we got any new data, add it to our result_df
if new_data:
    # Combine new data
    new_market_data = pd.concat(new_data, ignore_index=True)
    
    # Remove timezone info
    new_market_data['date'] = new_market_data['date'].dt.tz_localize(None)
    
    # Remove old data for these tickers from result_df
    result_df = result_df[~result_df['underlying_ticker'].isin(missing_tickers)]
    
    # Add new data
    new_result_df = pd.merge(
        base_df[base_df['underlying_ticker'].isin(missing_tickers)],
        new_market_data,
        on=['date', 'underlying_ticker'],
        how='left'
    )
    
    # Combine old and new data
    result_df = pd.concat([result_df, new_result_df], ignore_index=True)
    
    # Sort the DataFrame
    result_df = result_df.sort_values(['date', 'underlying_ticker'])
    
    # Save updated dataset
    result_df.to_csv('data/processed/day_aggs/base_df_with_market_data.csv', index=False)
    
    # Show updated statistics
    missing_data = result_df[result_df['close'].isna()]
    print("\nUpdated statistics:")
    print(f"Total rows: {len(result_df)}")
    print(f"Missing data rows: {len(missing_data)}")
    print(f"Missing data percentage: {(len(missing_data)/len(result_df))*100:.2f}%")
else:
    print("No new data was successfully fetched")

In [36]:
# Calculate the percentage of non-null values for each date
trading_day_counts = result_df.groupby('date')['close'].count()
total_tickers = result_df['underlying_ticker'].nunique()
trading_day_pct = trading_day_counts / total_tickers

# Consider a day a trading day if it has data for at least 50% of tickers
trading_days = trading_day_pct[trading_day_pct > 0.5].index

# Filter for only trading days
clean_df = result_df[result_df['date'].isin(trading_days)].copy()

# Save the cleaned dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

print("\nCleaned dataset summary:")
print(f"Original rows: {len(result_df)}")
print(f"Cleaned rows: {len(clean_df)}")
print(f"Missing data in cleaned dataset: {clean_df['close'].isna().sum()}")

# Show distribution of data by date
print("\nNumber of records per date in cleaned dataset:")
print(clean_df.groupby('date')['close'].count().head())


Cleaned dataset summary:
Original rows: 183000
Cleaned rows: 125000
Missing data in cleaned dataset: 3220

Number of records per date in cleaned dataset:
date
2024-07-01    972
2024-07-02    972
2024-07-03    972
2024-07-05    972
2024-07-08    972
Name: close, dtype: int64


In [32]:
clean_df.head()

,date,underlying_ticker,open,high,low,close,volume
0,2024-07-01,AA,40.021413,40.637890,39.315446,39.474537,3228000.0
1,2024-07-01,AAL,11.330000,11.410000,11.000000,11.040000,28624300.0
2,2024-07-01,AAOI,8.390000,8.660000,7.850000,8.080000,1778500.0
3,2024-07-01,AAP,61.909940,62.147368,58.813453,59.070667,1767600.0
4,2024-07-01,AAPL,211.611974,217.019756,211.442359,216.261475,60402900.0


In [41]:
# Convert window_start from nanoseconds to datetime and normalize to remove time
filtered_df['trading_date'] = pd.to_datetime(filtered_df['window_start'] / 1e9, unit='s').dt.normalize()

# Calculate daily option volume
daily_option_volume = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])['volume']
    .sum()
    .reset_index()
    .rename(columns={
        'trading_date': 'date',
        'volume': 'total_option_volume'
    })
)

# Ensure clean_df dates are normalized
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.normalize()

# Merge with clean_df (drop any existing option volume columns first)
option_cols = [col for col in clean_df.columns if 'option_volume' in col]
clean_df = clean_df.drop(columns=option_cols, errors='ignore')

clean_df = pd.merge(
    clean_df,
    daily_option_volume,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill any missing option volume with 0
clean_df['total_option_volume'] = clean_df['total_option_volume'].fillna(0)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display some statistics
print("Sample of data with option volumes:")
print(clean_df[['date', 'underlying_ticker', 'close', 'volume', 'total_option_volume']].head())

print("\nTop 10 days by option volume:")
print(
    clean_df.groupby('date')['total_option_volume']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

print("\nTop 10 tickers by total option volume:")
print(
    clean_df.groupby('underlying_ticker')['total_option_volume']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

Sample of data with option volumes:
        date underlying_ticker       close      volume  total_option_volume
0 2024-07-01                AA   39.474537   3228000.0              18243.0
1 2024-07-01               AAL   11.040000  28624300.0             103084.0
2 2024-07-01              AAOI    8.080000   1778500.0               2974.0
3 2024-07-01               AAP   59.070667   1767600.0               6723.0
4 2024-07-01              AAPL  216.261475  60402900.0            1432971.0

Top 10 days by option volume:
date
2024-12-20    40736562.0
2024-11-08    40670366.0
2024-11-07    39613245.0
2024-11-15    39067922.0
2024-11-06    38337638.0
2024-11-22    37215509.0
2024-12-06    37154362.0
2024-11-11    37138074.0
2024-12-18    36554656.0
2024-12-13    36424182.0
Name: total_option_volume, dtype: float64

Top 10 tickers by total option volume:
underlying_ticker
NVDA    482102884.0
TSLA    267744644.0
AAPL    105612376.0
AMD      73154011.0
AMZN     70015318.0
PLTR     66198497.0
SM

In [42]:
# First, let's calculate the HHI for each ticker-day
def calculate_hhi(group):
    total_volume = group['volume'].sum()
    if total_volume == 0:
        return 0
    market_shares = group['volume'] / total_volume
    return (market_shares ** 2).sum()

# Calculate HHI for each ticker-day
hhi_by_day = (
    filtered_df.groupby(['trading_date', 'underlying_ticker', 'ticker'])['volume']
    .sum()
    .reset_index()
    .groupby(['trading_date', 'underlying_ticker'])
    .apply(calculate_hhi)
    .reset_index()
    .rename(columns={0: 'option_hhi'})
)

# Normalize dates
hhi_by_day['trading_date'] = pd.to_datetime(hhi_by_day['trading_date']).dt.normalize()
hhi_by_day = hhi_by_day.rename(columns={'trading_date': 'date'})

# Merge HHI with clean_df
clean_df = pd.merge(
    clean_df,
    hhi_by_day,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill missing HHI values with 0
clean_df['option_hhi'] = clean_df['option_hhi'].fillna(0)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display some statistics
print("Sample of data with HHI:")
print(clean_df[['date', 'underlying_ticker', 'total_option_volume', 'option_hhi']].head())

print("\nTop 10 most concentrated days (highest average HHI):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]  # Filter for meaningful volume
    .groupby('date')['option_hhi']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

print("\nTop 10 most concentrated tickers (highest average HHI):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]  # Filter for meaningful volume
    .groupby('underlying_ticker')['option_hhi']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

# Calculate some summary statistics
print("\nHHI Summary Statistics (for days with significant volume):")
hhi_stats = clean_df[clean_df['total_option_volume'] > 1000]['option_hhi'].describe()
print(hhi_stats)

# Optional: Create volume-weighted HHI
clean_df['volume_weighted_hhi'] = clean_df['option_hhi'] * (clean_df['total_option_volume'] / clean_df.groupby('date')['total_option_volume'].transform('sum'))

print("\nTop 10 tickers by volume-weighted HHI:")
print(
    clean_df.groupby('underlying_ticker')['volume_weighted_hhi']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/3503309255.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_hhi)


Sample of data with HHI:
        date underlying_ticker  total_option_volume  option_hhi
0 2024-07-01                AA              18243.0    0.044966
1 2024-07-01               AAL             103084.0    0.042765
2 2024-07-01              AAOI               2974.0    0.148861
3 2024-07-01               AAP               6723.0    0.027098
4 2024-07-01              AAPL            1432971.0    0.033944

Top 10 most concentrated days (highest average HHI):
date
2024-09-10    0.143076
2024-09-17    0.141824
2024-10-10    0.141173
2024-09-30    0.139684
2024-08-16    0.139001
2024-09-20    0.138560
2024-10-01    0.138424
2024-09-25    0.137635
2024-09-05    0.136955
2024-10-23    0.136635
Name: option_hhi, dtype: float64

Top 10 most concentrated tickers (highest average HHI):
underlying_ticker
MUB      0.760964
AMCR     0.729887
MULN1    0.717769
EDR      0.649502
ARHS     0.624717
GSM      0.598321
YMM      0.595236
CRTO     0.589771
INFN     0.585690
BBD      0.585514
Name: option_h

In [43]:
# Calculate percentage distance from current price for each contract
def calculate_weighted_strike_distance(group):
    if len(group) == 0 or group['volume'].sum() == 0:
        return 0
    
    # Calculate percentage distance from current price for each contract
    pct_distances = (group['strike'] - group['close']) / group['close'] * 100
    
    # Calculate volume weights
    weights = group['volume'] / group['volume'].sum()
    
    # Return weighted average distance
    return (pct_distances * weights).sum()

# Calculate weighted strike distance for each ticker-day
strike_distances = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])
    .apply(calculate_weighted_strike_distance)
    .reset_index()
    .rename(columns={0: 'weighted_strike_distance'})
)

# Normalize dates
strike_distances['trading_date'] = pd.to_datetime(strike_distances['trading_date']).dt.normalize()
strike_distances = strike_distances.rename(columns={'trading_date': 'date'})

# Merge with clean_df
clean_df = pd.merge(
    clean_df,
    strike_distances,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill missing values with 0
clean_df['weighted_strike_distance'] = clean_df['weighted_strike_distance'].fillna(0)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display statistics
print("Sample of data with weighted strike distances:")
print(clean_df[['date', 'underlying_ticker', 'close', 'total_option_volume', 'weighted_strike_distance']].head())

print("\nTop 10 days by average absolute strike distance (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('date')['weighted_strike_distance']
    .mean()
    .abs()
    .sort_values(ascending=False)
    .head(10)
)

print("\nTop 10 tickers by average absolute strike distance (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('underlying_ticker')['weighted_strike_distance']
    .mean()
    .abs()
    .sort_values(ascending=False)
    .head(10)
)

# Calculate summary statistics
print("\nWeighted Strike Distance Summary Statistics (for days with significant volume):")
distance_stats = clean_df[clean_df['total_option_volume'] > 1000]['weighted_strike_distance'].describe()
print(distance_stats)

# Optional: Calculate separate stats for calls and puts
def calculate_weighted_strike_distance_by_type(group):
    if len(group) == 0 or group['volume'].sum() == 0:
        return pd.Series({'call_distance': 0, 'put_distance': 0})
    
    # Split into calls and puts
    calls = group[group['type'] == 'C']
    puts = group[group['type'] == 'P']
    
    # Calculate for calls
    if len(calls) > 0 and calls['volume'].sum() > 0:
        call_weights = calls['volume'] / calls['volume'].sum()
        call_distance = ((calls['strike'] - calls['close']) / calls['close'] * 100 * call_weights).sum()
    else:
        call_distance = 0
        
    # Calculate for puts
    if len(puts) > 0 and puts['volume'].sum() > 0:
        put_weights = puts['volume'] / puts['volume'].sum()
        put_distance = ((puts['strike'] - puts['close']) / puts['close'] * 100 * put_weights).sum()
    else:
        put_distance = 0
    
    return pd.Series({'call_distance': call_distance, 'put_distance': put_distance})

# Calculate separate distances for calls and puts
type_distances = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])
    .apply(calculate_weighted_strike_distance_by_type)
    .reset_index()
)

print("\nSeparate Call/Put Distance Statistics:")
print(type_distances.describe())

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/646860361.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_weighted_strike_distance)


Sample of data with weighted strike distances:
        date underlying_ticker       close  total_option_volume  \
0 2024-07-01                AA   39.474537              18243.0   
1 2024-07-01               AAL   11.040000             103084.0   
2 2024-07-01              AAOI    8.080000               2974.0   
3 2024-07-01               AAP   59.070667               6723.0   
4 2024-07-01              AAPL  216.261475            1432971.0   

   weighted_strike_distance  
0              48811.239887  
1              15258.147886  
2               4045.293617  
3              64496.290477  
4              71137.721162  

Top 10 days by average absolute strike distance (minimum volume filter):
date
2024-07-26    126972.298115
2024-12-13    125204.457336
2024-09-27    122304.772522
2024-08-09    121666.082618
2024-11-29    120491.806344
2024-08-30    118227.231546
2024-07-12    115888.611273
2024-07-05    115696.822217
2024-09-13    115039.986952
2024-11-01    114507.266995
Name: weigh

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/646860361.py:98: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_weighted_strike_distance_by_type)


In [44]:
print("Columns in clean_df:")
print(clean_df.columns.tolist())

# Show a sample with our key metrics
print("\nSample of clean_df with key metrics:")
print(clean_df[['date', 'underlying_ticker', 'close', 'total_option_volume', 'option_hhi', 'weighted_strike_distance']].head())

Columns in clean_df:
['date', 'underlying_ticker', 'open', 'high', 'low', 'close', 'volume', 'total_option_volume', 'option_hhi', 'volume_weighted_hhi', 'weighted_strike_distance']

Sample of clean_df with key metrics:
        date underlying_ticker       close  total_option_volume  option_hhi  \
0 2024-07-01                AA   39.474537              18243.0    0.044966   
1 2024-07-01               AAL   11.040000             103084.0    0.042765   
2 2024-07-01              AAOI    8.080000               2974.0    0.148861   
3 2024-07-01               AAP   59.070667               6723.0    0.027098   
4 2024-07-01              AAPL  216.261475            1432971.0    0.033944   

   weighted_strike_distance  
0              48811.239887  
1              15258.147886  
2               4045.293617  
3              64496.290477  
4              71137.721162  


In [47]:
import numpy as np

def calculate_log_moneyness_skew(group):
    if len(group) == 0 or group['volume'].sum() == 0:
        return 0
    
    # Calculate log-moneyness for each contract
    log_moneyness = np.log(group['strike'] / group['close'])
    
    # Split into calls and puts
    calls = group[group['type'] == 'C']
    puts = group[group['type'] == 'P']
    
    # Calculate weighted log-moneyness for calls
    if len(calls) > 0 and calls['volume'].sum() > 0:
        call_weights = calls['volume'] / calls['volume'].sum()
        call_weighted_moneyness = (np.log(calls['strike'] / calls['close']) * call_weights).sum()
    else:
        call_weighted_moneyness = 0
        
    # Calculate weighted log-moneyness for puts
    if len(puts) > 0 and puts['volume'].sum() > 0:
        put_weights = puts['volume'] / puts['volume'].sum()
        put_weighted_moneyness = (np.log(puts['strike'] / puts['close']) * put_weights).sum()
    else:
        put_weighted_moneyness = 0
    
    # Return the skew (put minus call)
    return put_weighted_moneyness - call_weighted_moneyness

# Calculate log-moneyness skew for each ticker-day
moneyness_skew = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])
    .apply(calculate_log_moneyness_skew)
    .reset_index()
    .rename(columns={0: 'log_moneyness_skew'})
)

# Normalize dates
moneyness_skew['trading_date'] = pd.to_datetime(moneyness_skew['trading_date']).dt.normalize()
moneyness_skew = moneyness_skew.rename(columns={'trading_date': 'date'})

# Merge with clean_df
clean_df = pd.merge(
    clean_df,
    moneyness_skew,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill missing values with 0
clean_df['log_moneyness_skew'] = clean_df['log_moneyness_skew'].fillna(0)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display statistics
print("Sample of data with log-moneyness skew:")
print(clean_df[['date', 'underlying_ticker', 'close', 'total_option_volume', 'log_moneyness_skew']].head())

print("\nTop 10 days by average absolute log-moneyness skew (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('date')['log_moneyness_skew']
    .mean()
    .abs()
    .sort_values(ascending=False)
    .head(10)
)

print("\nTop 10 tickers by average absolute log-moneyness skew (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('underlying_ticker')['log_moneyness_skew']
    .mean()
    .abs()
    .sort_values(ascending=False)
    .head(10)
)

# Calculate summary statistics
print("\nLog-Moneyness Skew Summary Statistics (for days with significant volume):")
skew_stats = clean_df[clean_df['total_option_volume'] > 1000]['log_moneyness_skew'].describe()
print(skew_stats)

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/4247650816.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_log_moneyness_skew)


Sample of data with log-moneyness skew:
        date underlying_ticker       close  total_option_volume  \
0 2024-07-01                AA   39.474537              18243.0   
1 2024-07-01               AAL   11.040000             103084.0   
2 2024-07-01              AAOI    8.080000               2974.0   
3 2024-07-01               AAP   59.070667               6723.0   
4 2024-07-01              AAPL  216.261475            1432971.0   

   log_moneyness_skew  
0           -0.685844  
1           -0.305484  
2            0.284867  
3           -0.815526  
4            1.070264  

Top 10 days by average absolute log-moneyness skew (minimum volume filter):
date
2024-12-18    0.828945
2024-12-19    0.811939
2024-08-07    0.665583
2024-09-06    0.637450
2024-08-05    0.566172
2024-12-23    0.485638
2024-08-02    0.481456
2024-09-05    0.480052
2024-12-12    0.466207
2024-07-18    0.449758
Name: log_moneyness_skew, dtype: float64

Top 10 tickers by average absolute log-moneyness skew (mini

In [49]:
def calculate_expiry_variance(group):
    if len(group) == 0 or group['volume'].sum() == 0:
        return pd.Series({
            'weighted_mean_dte': 0,
            'weighted_var_dte': 0
        })
    
    # Calculate days to expiration for each contract
    group = group.copy()  # Avoid SettingWithCopyWarning
    group['dte'] = (pd.to_datetime(group['expiration_date']) - pd.to_datetime(group['trading_date'])).dt.days
    
    # Calculate volume weights
    total_volume = group['volume'].sum()
    volume_weights = group['volume'] / total_volume
    
    # Calculate weighted mean DTE
    weighted_mean_dte = (group['dte'] * volume_weights).sum()
    
    # Calculate weighted variance
    squared_deviations = (group['dte'] - weighted_mean_dte) ** 2
    weighted_variance = (squared_deviations * volume_weights).sum()
    
    return pd.Series({
        'weighted_mean_dte': weighted_mean_dte,
        'weighted_var_dte': weighted_variance
    })

# Calculate expiry variance for each ticker-day
expiry_stats = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])
    .apply(calculate_expiry_variance)
    .reset_index()
)

# Normalize dates
expiry_stats['trading_date'] = pd.to_datetime(expiry_stats['trading_date']).dt.normalize()
expiry_stats = expiry_stats.rename(columns={'trading_date': 'date'})

# Merge with clean_df
clean_df = pd.merge(
    clean_df,
    expiry_stats,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill missing values with 0
clean_df['weighted_mean_dte'] = clean_df['weighted_mean_dte'].fillna(0)
clean_df['weighted_var_dte'] = clean_df['weighted_var_dte'].fillna(0)

# Add standard deviation column for easier interpretation
clean_df['weighted_std_dte'] = np.sqrt(clean_df['weighted_var_dte'])

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display statistics
print("Sample of data with expiry statistics:")
print(clean_df[['date', 'underlying_ticker', 'total_option_volume', 
                'weighted_mean_dte', 'weighted_std_dte']].head())

print("\nTop 10 tickers by average DTE spread (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('underlying_ticker')['weighted_std_dte']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

# Calculate summary statistics
print("\nDTE Statistics (for days with significant volume):")
dte_stats = clean_df[clean_df['total_option_volume'] > 1000][
    ['weighted_mean_dte', 'weighted_std_dte']
].describe()
print(dte_stats)

# Optional: Look at correlation between DTE spread and other metrics
print("\nCorrelations with DTE spread:")
correlation_cols = ['total_option_volume', 'option_hhi', 'log_moneyness_skew', 'weighted_std_dte']
correlations = clean_df[clean_df['total_option_volume'] > 1000][correlation_cols].corr()['weighted_std_dte']
print(correlations)

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/2539631142.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_expiry_variance)


Sample of data with expiry statistics:
        date underlying_ticker  total_option_volume  weighted_mean_dte  \
0 2024-07-01                AA              18243.0          62.026750   
1 2024-07-01               AAL             103084.0          65.054713   
2 2024-07-01              AAOI               2974.0          69.885676   
3 2024-07-01               AAP               6723.0          36.814517   
4 2024-07-01              AAPL            1432971.0          30.496198   

   weighted_std_dte  
0         95.056529  
1        106.963569  
2         79.813993  
3         89.482436  
4         76.720547  

Top 10 tickers by average DTE spread (minimum volume filter):
underlying_ticker
VALE    206.198355
POET    197.246402
BTG     196.104729
DNN     184.346450
EOSE    183.550141
GRAB    180.726872
TELL    179.236952
SBSW    178.097077
LAZR    176.317763
LAC     173.035344
Name: weighted_std_dte, dtype: float64

DTE Statistics (for days with significant volume):
       weighted_mean_d

In [50]:
clean_df.head()

,date,underlying_ticker,open,high,low,close,volume,total_option_volume,option_hhi,volume_weighted_hhi,weighted_strike_distance,log_moneyness_skew,weighted_mean_dte,weighted_var_dte,weighted_std_dte
0,2024-07-01,AA,40.021413,40.637890,39.315446,39.474537,3228000.0,18243.0,0.044966,0.000033,48811.239887,-0.685844,62.026750,9035.743625,95.056529
1,2024-07-01,AAL,11.330000,11.410000,11.000000,11.040000,28624300.0,103084.0,0.042765,0.000175,15258.147886,-0.305484,65.054713,11441.205050,106.963569
2,2024-07-01,AAOI,8.390000,8.660000,7.850000,8.080000,1778500.0,2974.0,0.148861,0.000018,4045.293617,0.284867,69.885676,6370.273413,79.813993
3,2024-07-01,AAP,61.909940,62.147368,58.813453,59.070667,1767600.0,6723.0,0.027098,0.000007,64496.290477,-0.815526,36.814517,8007.106307,89.482436
4,2024-07-01,AAPL,211.611974,217.019756,211.442359,216.261475,60402900.0,1432971.0,0.033944,0.001932,71137.721162,1.070264,30.496198,5886.042300,76.720547


In [52]:
# First calculate the raw O/S ratio for each ticker-day
def calculate_os_ratio(group):
    # Convert option volume to share equivalent (× 100)
    option_share_volume = group['option_volume'].sum() * 100
    # Get stock volume
    stock_volume = group['stock_volume'].iloc[0]
    # Calculate ratio
    return option_share_volume / stock_volume if stock_volume > 0 else 0

# Calculate daily O/S ratios
daily_os = (
    filtered_df.groupby(['trading_date', 'underlying_ticker'])['volume']
    .sum()
    .reset_index()
    .rename(columns={'volume': 'option_volume'})
)

# Merge with stock volume data
daily_os = pd.merge(
    daily_os,
    clean_df[['date', 'underlying_ticker', 'volume']].rename(columns={'volume': 'stock_volume'}),
    left_on=['trading_date', 'underlying_ticker'],
    right_on=['date', 'underlying_ticker'],
    how='left'
)

# Calculate raw O/S ratio
daily_os['os_ratio'] = daily_os.groupby('underlying_ticker').apply(calculate_os_ratio).reset_index(drop=True)

# Calculate 20-day moving average O/S ratio
daily_os['date'] = pd.to_datetime(daily_os['date'])
daily_os = daily_os.sort_values(['underlying_ticker', 'date'])
daily_os['os_ratio_ma20'] = daily_os.groupby('underlying_ticker')['os_ratio'].transform(
    lambda x: x.rolling(window=20, min_periods=5).mean()
)

# Calculate normalized O/S ratio
daily_os['normalized_os_ratio'] = daily_os['os_ratio'] / daily_os['os_ratio_ma20']

# Clean up and prepare for merge
os_metrics = daily_os[['date', 'underlying_ticker', 'os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']]

# Merge with clean_df
clean_df = pd.merge(
    clean_df,
    os_metrics,
    on=['date', 'underlying_ticker'],
    how='left'
)

# Fill missing values with 0
clean_df[['os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']] = clean_df[
    ['os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']
].fillna(0)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display statistics
print("Sample of data with O/S ratios:")
print(clean_df[['date', 'underlying_ticker', 'volume', 'total_option_volume', 
                'os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']].head())

print("\nTop 10 stocks by average O/S ratio (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('underlying_ticker')['os_ratio']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

/var/folders/s0/8mjnv1ts4lb9snv_c2g7gslh0000gn/T/ipykernel_34817/2356105829.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_os['os_ratio'] = daily_os.groupby('underlying_ticker').apply(calculate_os_ratio).reset_index(drop=True)


Sample of data with O/S ratios:
        date underlying_ticker      volume  total_option_volume    os_ratio  \
0 2024-07-01                AA   3228000.0              18243.0   70.113166   
1 2024-07-01               AAL  28624300.0             103084.0   53.479809   
2 2024-07-01              AAOI   1778500.0               2974.0   40.153725   
3 2024-07-01               AAP   1767600.0               6723.0   67.741967   
4 2024-07-01              AAPL  60402900.0            1432971.0  174.846532   

   os_ratio_ma20  normalized_os_ratio  
0            0.0                  0.0  
1            0.0                  0.0  
2            0.0                  0.0  
3            0.0                  0.0  
4            0.0                  0.0  

Top 10 stocks by average O/S ratio (minimum volume filter):
underlying_ticker
MVIS     224.045436
CLMT       9.371030
TPR        8.813288
GSM        6.319152
XRX        5.562419
MLCO       5.406618
MCHP       4.507057
NKLA1      4.028937
OPTT       3.7

In [56]:
import yfinance as yf

# First, let's get SPY data for market returns
spy = yf.download("SPY", 
                  start=clean_df['date'].min(),
                  end=clean_df['date'].max() + pd.Timedelta(days=30))  # Extra 30 days for forward returns

# Calculate forward returns for different horizons (e.g., 5, 10, 20 days)
def calculate_forward_returns(df, horizons=[5, 10, 20]):
    """
    Calculate forward returns for multiple horizons
    df: DataFrame with dates and tickers
    horizons: List of forward days to calculate returns for
    """
    # Calculate market returns for each horizon
    for n in horizons:
        spy['forward_ret_' + str(n)] = (
            spy['Close'].shift(-n) / spy['Close'] - 1
        )
    
    # For each stock and horizon
    for n in horizons:
        # Calculate stock's forward return
        df['forward_ret_' + str(n)] = df.groupby('underlying_ticker')['close'].transform(
            lambda x: x.shift(-n) / x - 1
        )
        
        # Calculate excess return
        df['excess_ret_' + str(n)] = (
            df['forward_ret_' + str(n)] - 
            spy.loc[df['date']]['forward_ret_' + str(n)].values
        )
        
    return df

# Function to create binary signals based on threshold
def create_binary_signals(df, threshold, horizons=[5, 10, 20]):
    """
    Create binary signals for significant moves
    threshold: Percentage threshold for significant moves (e.g., 0.02 for 2%)
    """
    for n in horizons:
        col_name = f'signal_{n}d_{int(threshold*100)}pct'
        df[col_name] = (
            (df['excess_ret_' + str(n)].abs() > threshold)
            .astype(int)
        )
    return df

# Apply the calculations
horizons = [5, 10, 20]  # Different time horizons to consider
thresholds = [0.02, 0.03, 0.05]  # Different thresholds to test

# Calculate returns and signals for all stocks at once
clean_df = calculate_forward_returns(clean_df, horizons)

# Create signals for different thresholds
for threshold in thresholds:
    clean_df = create_binary_signals(clean_df, threshold, horizons)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display class balance for different horizons and thresholds
print("\nClass balance for different horizons and thresholds:")
for n in horizons:
    for threshold in thresholds:
        col_name = f'signal_{n}d_{int(threshold*100)}pct'
        signal_dist = clean_df[col_name].value_counts(normalize=True)
        print(f"\n{n}-day, {threshold*100}% threshold:")
        print(signal_dist)
        
# Calculate average signal rate by ticker
print("\nTop 10 stocks by signal frequency (20-day, 2% threshold):")
signal_col = 'signal_20d_2pct'
print(
    clean_df.groupby('underlying_ticker')[signal_col]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

# Optional: Show correlation with our option metrics
print("\nCorrelation with option metrics (20-day, 2% threshold):")
correlation_cols = ['total_option_volume', 'option_hhi', 'log_moneyness_skew', 
                   'weighted_std_dte', 'normalized_os_ratio', signal_col]
correlations = clean_df[correlation_cols].corr()[signal_col]
print(correlations)

[*********************100%***********************]  1 of 1 completed



Class balance for different horizons and thresholds:

5-day, 2.0% threshold:
signal_5d_2pct
1    0.616856
0    0.383144
Name: proportion, dtype: float64

5-day, 3.0% threshold:
signal_5d_3pct
0    0.507632
1    0.492368
Name: proportion, dtype: float64

5-day, 5.0% threshold:
signal_5d_5pct
0    0.684864
1    0.315136
Name: proportion, dtype: float64

10-day, 2.0% threshold:
signal_10d_2pct
1    0.686448
0    0.313552
Name: proportion, dtype: float64

10-day, 3.0% threshold:
signal_10d_3pct
1    0.591312
0    0.408688
Name: proportion, dtype: float64

10-day, 5.0% threshold:
signal_10d_5pct
0    0.564944
1    0.435056
Name: proportion, dtype: float64

20-day, 2.0% threshold:
signal_20d_2pct
1    0.68244
0    0.31756
Name: proportion, dtype: float64

20-day, 3.0% threshold:
signal_20d_3pct
1    0.620032
0    0.379968
Name: proportion, dtype: float64

20-day, 5.0% threshold:
signal_20d_5pct
1    0.506208
0    0.493792
Name: proportion, dtype: float64

Top 10 stocks by signal frequency (

In [57]:
# Basic statistics for normalized_os_ratio
print("Normalized O/S Ratio Statistics:")
print(clean_df['normalized_os_ratio'].describe())

# Check for NaN values
print("\nNaN count in normalized_os_ratio:", clean_df['normalized_os_ratio'].isna().sum())

# Check for infinite values
print("\nInfinite values in normalized_os_ratio:", 
      clean_df['normalized_os_ratio'].isin([np.inf, -np.inf]).sum())

# Look at some extreme values
print("\nTop 10 highest normalized_os_ratio values:")
print(clean_df.nlargest(10, 'normalized_os_ratio')[
    ['date', 'underlying_ticker', 'os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']
])

print("\nSample of rows where normalized_os_ratio is NaN:")
print(clean_df[clean_df['normalized_os_ratio'].isna()][
    ['date', 'underlying_ticker', 'os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']
].head())

# Check the components
print("\nStatistics for components:")
print("\nos_ratio:")
print(clean_df['os_ratio'].describe())
print("\nos_ratio_ma20:")
print(clean_df['os_ratio_ma20'].describe())

# Check for zero values in ma20 (which would cause division by zero)
print("\nCount of zero values in os_ratio_ma20:", (clean_df['os_ratio_ma20'] == 0).sum())

# Distribution of values
print("\nValue ranges for normalized_os_ratio:")
ranges = [-np.inf, 0, 1, 2, 5, 10, np.inf]
print(pd.cut(clean_df['normalized_os_ratio'], bins=ranges).value_counts().sort_index())

Normalized O/S Ratio Statistics:
count    125000.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: normalized_os_ratio, dtype: float64

NaN count in normalized_os_ratio: 0

Infinite values in normalized_os_ratio: 0

Top 10 highest normalized_os_ratio values:
        date underlying_ticker    os_ratio  os_ratio_ma20  normalized_os_ratio
0 2024-07-01                AA   70.113166            0.0                  0.0
1 2024-07-01               AAL   53.479809            0.0                  0.0
2 2024-07-01              AAOI   40.153725            0.0                  0.0
3 2024-07-01               AAP   67.741967            0.0                  0.0
4 2024-07-01              AAPL  174.846532            0.0                  0.0
5 2024-07-01              ABBV   49.140360            0.0                  0.0
6 2024-07-01              ABNB  134.106093            0.0                  0.0
7 2024-07-01             

In [59]:
# First calculate the raw O/S ratio for each ticker-day
def calculate_os_ratio(group):
    # Convert option volume to share equivalent (× 100)
    option_share_volume = group['total_option_volume'] * 100
    # Get stock volume
    stock_volume = group['volume']
    # Calculate ratio
    return option_share_volume / stock_volume if stock_volume > 0 else 0

# Calculate daily O/S ratios
clean_df['os_ratio'] = clean_df.apply(
    lambda row: calculate_os_ratio(row), 
    axis=1
)

# Sort by date for proper rolling calculations
clean_df = clean_df.sort_values(['underlying_ticker', 'date'])

# Calculate 20-day moving average O/S ratio
clean_df['os_ratio_ma20'] = clean_df.groupby('underlying_ticker')['os_ratio'].transform(
    lambda x: x.rolling(window=20, min_periods=5).mean()
)

# Calculate normalized O/S ratio
clean_df['normalized_os_ratio'] = clean_df.apply(
    lambda row: row['os_ratio'] / row['os_ratio_ma20'] if row['os_ratio_ma20'] > 0 else 0,
    axis=1
)

# Save updated dataset
clean_df.to_csv('data/processed/day_aggs/clean_market_data.csv', index=False)

# Display statistics
print("Sample of data with O/S ratios:")
print(clean_df[['date', 'underlying_ticker', 'volume', 'total_option_volume', 
                'os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']].head())

print("\nTop 10 stocks by average O/S ratio (minimum volume filter):")
print(
    clean_df[clean_df['total_option_volume'] > 1000]
    .groupby('underlying_ticker')['os_ratio']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

print("\nO/S Ratio Statistics:")
print(clean_df[clean_df['total_option_volume'] > 1000][
    ['os_ratio', 'os_ratio_ma20', 'normalized_os_ratio']
].describe())

# Distribution of normalized ratios
print("\nDistribution of normalized O/S ratios (for significant volume):")
ranges = [0, 0.5, 1, 2, 5, 10, float('inf')]
print(pd.cut(
    clean_df[clean_df['total_option_volume'] > 1000]['normalized_os_ratio'],
    bins=ranges
).value_counts().sort_index())

Sample of data with O/S ratios:
           date underlying_ticker     volume  total_option_volume  os_ratio  \
0    2024-07-01                AA  3228000.0              18243.0  0.565149   
1000 2024-07-02                AA  4039500.0              35060.0  0.867929   
2000 2024-07-03                AA  4074500.0              32198.0  0.790232   
3000 2024-07-05                AA  2670200.0              14792.0  0.553966   
4000 2024-07-08                AA  3001000.0              12452.0  0.414928   

      os_ratio_ma20  normalized_os_ratio  
0               NaN             0.000000  
1000            NaN             0.000000  
2000            NaN             0.000000  
3000            NaN             0.000000  
4000       0.638441             0.649909  

Top 10 stocks by average O/S ratio (minimum volume filter):
underlying_ticker
MATV    3.961949
CRTO    3.508786
SIRI    3.097998
NFLX    2.980975
META    2.723343
COST    2.646241
CVNA    2.486378
GME     2.329859
TSLA    2.273751
COI

In [60]:
# Create feature set with our key metrics
features = [
    'total_option_volume',  # Overall option activity
    'option_hhi',          # Concentration of activity
    'log_moneyness_skew',  # Put vs call strike positioning
    'weighted_std_dte',    # Time horizon spread
    'normalized_os_ratio'  # Option vs stock volume
]

# Target variables (using 3% threshold at different horizons)
targets = [
    'signal_5d_3pct',
    'signal_10d_3pct', 
    'signal_20d_3pct'
]

# Create datasets for each prediction horizon
for target in targets:
    # Filter for minimum volume and valid ratios
    model_data = clean_df[
        (clean_df['total_option_volume'] > 1000) &  # Significant option activity
        (clean_df['os_ratio_ma20'] > 0)            # Valid O/S ratio
    ].copy()
    
    # Add any derived features
    model_data['log_option_volume'] = np.log1p(model_data['total_option_volume'])
    
    print(f"\nDataset for {target}:")
    print("Shape:", model_data[features + [target]].shape)
    print("\nClass balance:")
    print(model_data[target].value_counts(normalize=True))
    
    print("\nFeature correlations with target:")
    correlations = model_data[features + [target]].corr()[target].sort_values(ascending=False)
    print(correlations)
    
    # Save modeling dataset
    model_data[features + [target] + ['date', 'underlying_ticker']].to_csv(
        f'data/processed/model_data_{target}.csv',
        index=False
    )

# Display sample of final modeling data
print("\nSample of modeling data:")
print(model_data[features + targets].head())


Dataset for signal_5d_3pct:
Shape: (91904, 6)

Class balance:
signal_5d_3pct
1    0.508814
0    0.491186
Name: proportion, dtype: float64

Feature correlations with target:
signal_5d_3pct         1.000000
total_option_volume    0.016676
normalized_os_ratio    0.011137
weighted_std_dte       0.004773
option_hhi            -0.014807
log_moneyness_skew    -0.038395
Name: signal_5d_3pct, dtype: float64

Dataset for signal_10d_3pct:
Shape: (91904, 6)

Class balance:
signal_10d_3pct
1    0.607144
0    0.392856
Name: proportion, dtype: float64

Feature correlations with target:
signal_10d_3pct        1.000000
total_option_volume    0.010458
normalized_os_ratio    0.008902
option_hhi             0.002206
log_moneyness_skew    -0.000957
weighted_std_dte      -0.010353
Name: signal_10d_3pct, dtype: float64

Dataset for signal_20d_3pct:
Shape: (91904, 6)

Class balance:
signal_20d_3pct
1    0.632704
0    0.367296
Name: proportion, dtype: float64

Feature correlations with target:
signal_20d_3pct

In [69]:
# Feature Engineering
print("Adding enhanced features...")

# 1. Interaction terms
model_data['volume_skew'] = model_data['total_option_volume'] * model_data['log_moneyness_skew']
model_data['volume_spread'] = model_data['total_option_volume'] * model_data['weighted_std_dte']
model_data['concentration_ratio'] = model_data['normalized_os_ratio'] * model_data['option_hhi']

# 2. Temporal features
# Sort for proper rolling calculations
model_data = model_data.sort_values(['underlying_ticker', 'date'])

# Rolling statistics
model_data['rolling_vol'] = model_data.groupby('underlying_ticker')['total_option_volume'].transform(
    lambda x: x.rolling(window=5, min_periods=1).std()
)
model_data['vol_change'] = model_data.groupby('underlying_ticker')['total_option_volume'].transform(
    lambda x: x.pct_change()
)

# 3. Relative measures
model_data['skew_trend'] = model_data.groupby('underlying_ticker')['log_moneyness_skew'].transform(
    lambda x: x.rolling(window=5, min_periods=1).mean()
)
model_data['os_ratio_trend'] = model_data.groupby('underlying_ticker')['normalized_os_ratio'].transform(
    lambda x: x.rolling(window=5, min_periods=1).mean()
)

# Updated feature list
features = [
    # Original features
    'total_option_volume',
    'option_hhi',
    'log_moneyness_skew',
    'weighted_std_dte',
    'normalized_os_ratio',
    'log_option_volume',
    # Interaction features
    'volume_skew',
    'volume_spread',
    'concentration_ratio',
    # Temporal features
    'rolling_vol',
    'vol_change',
    'skew_trend',
    'os_ratio_trend'
]

# Handle any NaN values from rolling calculations
model_data = model_data.fillna(0)

# Model Evaluation
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# Setup
target = 'signal_5d_3pct'

# Prepare data
model_data = model_data.sort_values('date')
X = model_data[features].values
y = model_data[target].values
dates = model_data['date'].values

# Initialize models
models = {
    'Logistic Regression': LogisticRegression(
        C=1.0,
        class_weight='balanced',
        random_state=42,
        max_iter=1000
    ),
    'Linear SVM': LinearSVC(
        C=1.0,
        class_weight='balanced',
        random_state=42,
        max_iter=1000,
        dual='auto'
    ),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(50, 25),
        max_iter=1000,
        early_stopping=True,
        random_state=42
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=6,
        class_weight='balanced',
        random_state=42
    )
}

# Time series cross validation
tscv = TimeSeriesSplit(n_splits=5)
scaler = StandardScaler()

# Results storage
all_results = {}

# Evaluate each model
for name, model in models.items():
    print(f"\nEvaluating {name}")
    fold_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        
        # Handle different model types for probabilities
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
            auc_score = roc_auc_score(y_val, y_pred_proba)
        else:  # For LinearSVC
            y_pred_score = model.decision_function(X_val_scaled)
            auc_score = roc_auc_score(y_val, y_pred_score)
        
        fold_metrics.append({
            'fold': fold + 1,
            'train_dates': (dates[train_idx].min(), dates[train_idx].max()),
            'val_dates': (dates[val_idx].min(), dates[val_idx].max()),
            'roc_auc': auc_score,
            'report': classification_report(y_val, y_pred, output_dict=True)
        })
    
    all_results[name] = fold_metrics

# Print results
for name, results in all_results.items():
    print(f"\n=== {name} Results ===")
    avg_auc = np.mean([fold['roc_auc'] for fold in results])
    print(f"Average ROC AUC: {avg_auc:.4f}")
    
    # Print last fold details
    last_fold = results[-1]
    print(f"\nLast Fold Results:")
    print(f"Train: {last_fold['train_dates'][0]} to {last_fold['train_dates'][1]}")
    print(f"Val: {last_fold['val_dates'][0]} to {last_fold['val_dates'][1]}")
    print("\nClassification Report:")
    print(pd.DataFrame(last_fold['report']).transpose())

# Feature importance for Random Forest
if 'Random Forest' in models:
    rf_model = models['Random Forest']
    importance = pd.DataFrame({
        'feature': features,
        'importance': rf_model.feature_importances_
    })
    print("\nFeature Importance (Random Forest):")
    print(importance.sort_values('importance', ascending=False))

Adding enhanced features...

Evaluating Logistic Regression

Evaluating Linear SVM

Evaluating Neural Network

Evaluating Random Forest

=== Logistic Regression Results ===
Average ROC AUC: 0.5428

Last Fold Results:
Train: 2024-07-08T00:00:00.000000000 to 2024-11-26T00:00:00.000000000
Val: 2024-11-26T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.559898  0.474317  0.513567   8819.000000
1              0.409126  0.493998  0.447574   6498.000000
accuracy       0.482666  0.482666  0.482666      0.482666
macro avg      0.484512  0.484157  0.480570  15317.000000
weighted avg   0.495935  0.482666  0.485570  15317.000000

=== Linear SVM Results ===
Average ROC AUC: 0.5428

Last Fold Results:
Train: 2024-07-08T00:00:00.000000000 to 2024-11-26T00:00:00.000000000
Val: 2024-11-26T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-s

In [70]:
# Add more skew-related features
model_data['skew_acceleration'] = model_data.groupby('underlying_ticker')['skew_trend'].transform(
    lambda x: x.pct_change()
)

model_data['skew_volatility'] = model_data.groupby('underlying_ticker')['log_moneyness_skew'].transform(
    lambda x: x.rolling(window=5).std()
)

In [71]:
# Add multiple lookback periods
for window in [3, 5, 10]:
    model_data[f'skew_trend_{window}d'] = model_data.groupby('underlying_ticker')['log_moneyness_skew'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

In [72]:
# Add non-linear transformations of top features
model_data['skew_trend_squared'] = model_data['skew_trend'] ** 2
model_data['skew_trend_sign'] = np.sign(model_data['skew_trend'])

In [74]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier

# Setup for both horizons
targets = ['signal_5d_3pct', 'signal_10d_3pct']
features = [
    # Original features
    'total_option_volume', 'option_hhi', 'log_moneyness_skew',
    'weighted_std_dte', 'normalized_os_ratio', 'log_option_volume',
    # Interaction features
    'volume_skew', 'volume_spread', 'concentration_ratio',
    # Temporal features
    'rolling_vol', 'vol_change', 'skew_trend', 'os_ratio_trend',
    # New skew features
    'skew_acceleration', 'skew_volatility',
    'skew_trend_3d', 'skew_trend_5d', 'skew_trend_10d',
    'skew_trend_squared', 'skew_trend_sign'
]

# Drop rows with NaN values
print("Original shape:", model_data.shape)
model_data = model_data.dropna(subset=features + targets)
print("Shape after dropping NaNs:", model_data.shape)

# Initialize model with slightly larger architecture
model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    max_iter=1000,
    early_stopping=True,
    random_state=42,
    learning_rate_init=0.001,
    batch_size=256
)

# Evaluate for each horizon
for target in targets:
    print(f"\n=== Evaluating Neural Network for {target} ===")
    
    # Prepare data
    model_data = model_data.sort_values('date')
    X = model_data[features].values
    y = model_data[target].values
    dates = model_data['date'].values
    
    # Time series cross validation
    tscv = TimeSeriesSplit(n_splits=5)
    scaler = StandardScaler()
    
    # Results storage
    fold_metrics = []
    
    # Evaluate each fold
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Scale features
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        # Train and predict
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
        
        # Store metrics
        fold_metrics.append({
            'fold': fold + 1,
            'train_dates': (dates[train_idx].min(), dates[train_idx].max()),
            'val_dates': (dates[val_idx].min(), dates[val_idx].max()),
            'roc_auc': roc_auc_score(y_val, y_pred_proba),
            'report': classification_report(y_val, y_pred, output_dict=True)
        })
    
    # Print results
    print(f"\nResults for {target}:")
    avg_auc = np.mean([fold['roc_auc'] for fold in fold_metrics])
    print(f"Average ROC AUC: {avg_auc:.4f}")
    
    # Print last fold details
    last_fold = fold_metrics[-1]
    print(f"\nLast Fold Results:")
    print(f"Train: {last_fold['train_dates'][0]} to {last_fold['train_dates'][1]}")
    print(f"Val: {last_fold['val_dates'][0]} to {last_fold['val_dates'][1]}")
    print("\nClassification Report:")
    print(pd.DataFrame(last_fold['report']).transpose())
    
    # Print feature correlations with target
    correlations = pd.DataFrame({
        'feature': features,
        'correlation': [np.corrcoef(model_data[f], model_data[target])[0,1] for f in features]
    })
    print("\nTop 10 Feature Correlations:")
    print(correlations.sort_values('correlation', key=abs, ascending=False).head(10))

Original shape: (91904, 48)
Shape after dropping NaNs: (88000, 48)

=== Evaluating Neural Network for signal_5d_3pct ===

Results for signal_5d_3pct:
Average ROC AUC: 0.5659

Last Fold Results:
Train: 2024-07-12T00:00:00.000000000 to 2024-11-27T00:00:00.000000000
Val: 2024-11-27T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.620064  0.477067  0.539247   8525.000000
1              0.450105  0.594203  0.512212   6141.000000
accuracy       0.526115  0.526115  0.526115      0.526115
macro avg      0.535084  0.535635  0.525730  14666.000000
weighted avg   0.548898  0.526115  0.527927  14666.000000

Top 10 Feature Correlations:
                feature  correlation
17       skew_trend_10d    -0.076069
14      skew_volatility    -0.061494
11           skew_trend    -0.061386
16        skew_trend_5d    -0.061386
19      skew_trend_sign    -0.058722
15        skew_trend_3d    -0.052489
2    lo

In [75]:
# Add more sophisticated feature interactions
print("Adding enhanced features...")

# 1. Skew-based ratios and differences
model_data['skew_trend_ratio'] = model_data['skew_trend_10d'] / model_data['skew_trend_5d']
model_data['skew_momentum'] = model_data['skew_trend_10d'] - model_data['skew_trend_5d']
model_data['skew_vol_interaction'] = model_data['skew_volatility'] * model_data['skew_trend_10d']

# 2. Volume-skew interactions
model_data['vol_skew_ratio'] = model_data['skew_volatility'] / model_data['skew_trend']
model_data['vol_weighted_skew'] = model_data['total_option_volume'] * model_data['skew_trend']

# 3. Trend acceleration
model_data['skew_acceleration_3d'] = model_data.groupby('underlying_ticker')['skew_trend_3d'].transform(
    lambda x: x.pct_change()
)
model_data['skew_acceleration_5d'] = model_data.groupby('underlying_ticker')['skew_trend_5d'].transform(
    lambda x: x.pct_change()
)

# Updated feature list
features = [
    # Original features
    'total_option_volume', 'option_hhi', 'log_moneyness_skew',
    'weighted_std_dte', 'normalized_os_ratio', 'log_option_volume',
    # Interaction features
    'volume_skew', 'volume_spread', 'concentration_ratio',
    # Temporal features
    'rolling_vol', 'vol_change', 'skew_trend', 'os_ratio_trend',
    # Skew features
    'skew_acceleration', 'skew_volatility',
    'skew_trend_3d', 'skew_trend_5d', 'skew_trend_10d',
    'skew_trend_squared', 'skew_trend_sign',
    # New features
    'skew_trend_ratio', 'skew_momentum', 'skew_vol_interaction',
    'vol_skew_ratio', 'vol_weighted_skew',
    'skew_acceleration_3d', 'skew_acceleration_5d'
]

# Drop rows with NaN values
print("Original shape:", model_data.shape)
model_data = model_data.dropna(subset=features + ['signal_5d_3pct'])
print("Shape after dropping NaNs:", model_data.shape)

# Try a deeper network with dropout and regularization
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.neural_network import MLPClassifier

# Initialize model with enhanced architecture
model = MLPClassifier(
    hidden_layer_sizes=(200, 100, 50),  # Deeper network
    max_iter=2000,                      # More iterations
    early_stopping=True,
    random_state=42,
    learning_rate_init=0.0005,          # Lower learning rate
    batch_size=128,                     # Smaller batch size
    alpha=0.0001,                       # L2 regularization
    activation='relu',
    solver='adam',
    validation_fraction=0.15,           # Larger validation set
    n_iter_no_change=20                 # More patience
)

# Prepare data
X = model_data[features].values
y = model_data['signal_5d_3pct'].values
dates = model_data['date'].values

# Time series cross validation
tscv = TimeSeriesSplit(n_splits=5)
scaler = StandardScaler()

# Results storage
fold_metrics = []

# Evaluate each fold
for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
    print(f"\nTraining fold {fold+1}/5...")
    
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    # Scale features
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    # Train and predict
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_val_scaled)
    y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
    
    # Store metrics
    fold_metrics.append({
        'fold': fold + 1,
        'train_dates': (dates[train_idx].min(), dates[train_idx].max()),
        'val_dates': (dates[val_idx].min(), dates[val_idx].max()),
        'roc_auc': roc_auc_score(y_val, y_pred_proba),
        'report': classification_report(y_val, y_pred, output_dict=True)
    })

# Print results
print("\nOverall Results:")
avg_auc = np.mean([fold['roc_auc'] for fold in fold_metrics])
print(f"Average ROC AUC: {avg_auc:.4f}")

# Print last fold details
last_fold = fold_metrics[-1]
print(f"\nLast Fold Results:")
print(f"Train: {last_fold['train_dates'][0]} to {last_fold['train_dates'][1]}")
print(f"Val: {last_fold['val_dates'][0]} to {last_fold['val_dates'][1]}")
print("\nClassification Report:")
print(pd.DataFrame(last_fold['report']).transpose())

# Feature importance analysis
correlations = pd.DataFrame({
    'feature': features,
    'correlation': [np.corrcoef(model_data[f], model_data['signal_5d_3pct'])[0,1] for f in features]
})
print("\nTop 15 Feature Correlations:")
print(correlations.sort_values('correlation', key=abs, ascending=False).head(15))

Adding enhanced features...
Original shape: (88000, 55)
Shape after dropping NaNs: (87024, 55)

Training fold 1/5...

Training fold 2/5...

Training fold 3/5...

Training fold 4/5...

Training fold 5/5...

Overall Results:
Average ROC AUC: 0.5565

Last Fold Results:
Train: 2024-07-15T00:00:00.000000000 to 2024-11-27T00:00:00.000000000
Val: 2024-11-27T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.589177  0.387078  0.467208   8466.000000
1              0.419705  0.621563  0.501068   6038.000000
accuracy       0.484694  0.484694  0.484694      0.484694
macro avg      0.504441  0.504321  0.484138  14504.000000
weighted avg   0.518626  0.484694  0.481304  14504.000000

Top 15 Feature Correlations:
                 feature  correlation
17        skew_trend_10d    -0.075955
22  skew_vol_interaction    -0.062571
14       skew_volatility    -0.062239
16         skew_trend_5d    -0.061153
11 

In [77]:
def create_enhanced_features(df):
    """Create enhanced feature set from base dataframe."""
    df = df.copy()
    
    # 1. Non-linear transformations of top features
    df['skew_trend_10d_squared'] = df['skew_trend_10d'] ** 2
    df['skew_trend_10d_cubed'] = df['skew_trend_10d'] ** 3
    df['skew_volatility_squared'] = df['skew_volatility'] ** 2
    
    # 2. Exponential moving averages
    for window in [3, 5, 10]:
        df[f'skew_ema_{window}d'] = df.groupby('underlying_ticker')['log_moneyness_skew'].transform(
            lambda x: x.ewm(span=window, adjust=False).mean()
        )
    
    # 3. RSI for skew
    df['skew_rsi'] = df.groupby('underlying_ticker')['log_moneyness_skew'].transform(
        lambda x: (
            100 - (100 / (1 + (
                x.diff().clip(lower=0).rolling(window=14).mean() /
                -x.diff().clip(upper=0).rolling(window=14).mean()
            )))
        )
    )
    
    return df

def run_experiment(model_data, features, model, target='signal_5d_3pct', n_splits=5):
    """Run complete experiment with cross validation."""
    print(f"\nRunning experiment with {len(features)} features")
    
    # Drop NaN values first
    valid_data = model_data.dropna(subset=features + [target])
    print(f"Shape after dropping NaNs: {valid_data.shape}")
    
    # Prepare data
    X = valid_data[features].values
    y = valid_data[target].values
    dates = valid_data['date'].values
    
    # Time series cross validation
    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = []
    
    # Evaluate each fold
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        print(f"Training fold {fold+1}/{n_splits}...")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)
        
        # Train and predict
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
        
        # Store metrics
        fold_metrics.append({
            'fold': fold + 1,
            'train_dates': (dates[train_idx].min(), dates[train_idx].max()),
            'val_dates': (dates[val_idx].min(), dates[val_idx].max()),
            'roc_auc': roc_auc_score(y_val, y_pred_proba),
            'report': classification_report(y_val, y_pred, output_dict=True)
        })
    
    # Print results
    avg_auc = np.mean([m['roc_auc'] for m in fold_metrics])
    print(f"\nAverage ROC AUC: {avg_auc:.4f}")
    
    # Print last fold details
    last_fold = fold_metrics[-1]
    print(f"\nLast Fold Results:")
    print(f"Train: {last_fold['train_dates'][0]} to {last_fold['train_dates'][1]}")
    print(f"Val: {last_fold['val_dates'][0]} to {last_fold['val_dates'][1]}")
    print("\nClassification Report:")
    print(pd.DataFrame(last_fold['report']).transpose())
    
    return fold_metrics

# Create enhanced features
print("Creating enhanced features...")
model_data = create_enhanced_features(model_data)

# Define feature sets
base_features = [
    'skew_trend_10d', 'skew_volatility', 'log_moneyness_skew',
    'log_option_volume', 'volume_spread'
]

enhanced_features = base_features + [
    'skew_trend_10d_squared', 'skew_trend_10d_cubed', 
    'skew_volatility_squared',
    'skew_ema_3d', 'skew_ema_5d', 'skew_ema_10d',
    'skew_rsi'
]

# Define models to test
models = {
    'Simple NN': MLPClassifier(
        hidden_layer_sizes=(50, 25),
        max_iter=1000,
        early_stopping=True,
        random_state=42,
        learning_rate_init=0.001,
        batch_size=256,
        alpha=0.0005,
        activation='tanh',
        solver='adam'
    ),
    'Deep NN': MLPClassifier(
        hidden_layer_sizes=(200, 100, 50),
        max_iter=2000,
        early_stopping=True,
        random_state=42,
        learning_rate_init=0.0005,
        batch_size=128,
        alpha=0.0001,
        activation='relu',
        solver='adam'
    )
}

# Run experiments
results = {}
for model_name, model in models.items():
    print(f"\n=== Testing {model_name} ===")
    
    # Test with base features
    print("\nTesting with base features:")
    base_results = run_experiment(model_data, base_features, model)
    
    # Test with enhanced features
    print("\nTesting with enhanced features:")
    enhanced_results = run_experiment(model_data, enhanced_features, model)
    
    results[model_name] = {
        'base': base_results,
        'enhanced': enhanced_results
    }

# Print summary
print("\n=== Summary ===")
for model_name, model_results in results.items():
    print(f"\n{model_name}:")
    print(f"Base features AUC: {np.mean([m['roc_auc'] for m in model_results['base']]):.4f}")
    print(f"Enhanced features AUC: {np.mean([m['roc_auc'] for m in model_results['enhanced']]):.4f}")

Creating enhanced features...

=== Testing Simple NN ===

Testing with base features:

Running experiment with 5 features
Shape after dropping NaNs: (87024, 62)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...

Average ROC AUC: 0.5550

Last Fold Results:
Train: 2024-07-15T00:00:00.000000000 to 2024-11-27T00:00:00.000000000
Val: 2024-11-27T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.593040  0.336168  0.429099   8466.000000
1              0.420917  0.676549  0.518961   6038.000000
accuracy       0.477868  0.477868  0.477868      0.477868
macro avg      0.506979  0.506358  0.474030  14504.000000
weighted avg   0.521386  0.477868  0.466508  14504.000000

Testing with enhanced features:

Running experiment with 12 features
Shape after dropping NaNs: (73402, 62)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Trainin

In [78]:
# 1. Add more sophisticated features
def create_advanced_features(df):
    df = df.copy()
    
    # Volatility-adjusted skew metrics
    df['vol_adj_skew'] = df['log_moneyness_skew'] / df['rolling_vol']
    df['vol_adj_skew_trend'] = df['skew_trend_10d'] / df['rolling_vol']
    
    # Momentum indicators
    df['skew_momentum_1d'] = df.groupby('underlying_ticker')['log_moneyness_skew'].diff()
    df['skew_momentum_5d'] = df.groupby('underlying_ticker')['log_moneyness_skew'].diff(5)
    
    # Moving average crossovers
    df['skew_ma_cross'] = df['skew_ema_3d'] - df['skew_ema_10d']
    df['skew_ma_cross_signal'] = np.sign(df['skew_ma_cross'])
    
    return df

# 2. Define optimized feature set
optimized_features = [
    # Core features (best performers)
    'skew_trend_10d', 'skew_volatility', 
    'vol_adj_skew', 'vol_adj_skew_trend',
    
    # Momentum features
    'skew_momentum_1d', 'skew_momentum_5d',
    'skew_ma_cross', 'skew_ma_cross_signal',
    
    # Technical indicators
    'skew_rsi', 'skew_ema_3d', 'skew_ema_10d'
]

# 3. Optimized model architecture
optimized_nn = MLPClassifier(
    hidden_layer_sizes=(100, 50, 25),  # Balanced architecture
    max_iter=1500,                     # More iterations
    early_stopping=True,
    random_state=42,
    learning_rate_init=0.0008,         # Balanced learning rate
    batch_size=192,                    # Balanced batch size
    alpha=0.0003,                      # Balanced regularization
    activation='relu',
    solver='adam',
    validation_fraction=0.2,           # Larger validation set
    n_iter_no_change=15               # More patience
)

# Run new experiment
print("Creating advanced features...")
model_data = create_advanced_features(model_data)

print("\n=== Testing Optimized Model ===")
optimized_results = run_experiment(model_data, optimized_features, optimized_nn)

# Print final results
print("\n=== Final Results ===")
print(f"Optimized Model AUC: {np.mean([m['roc_auc'] for m in optimized_results]):.4f}")

Creating advanced features...

=== Testing Optimized Model ===

Running experiment with 11 features
Shape after dropping NaNs: (73402, 68)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...

Average ROC AUC: 0.5544

Last Fold Results:
Train: 2024-08-02T00:00:00.000000000 to 2024-12-03T00:00:00.000000000
Val: 2024-12-03T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.634567  0.435795  0.516725   7515.000000
1              0.400452  0.600254  0.480407   4718.000000
accuracy       0.499223  0.499223  0.499223      0.499223
macro avg      0.517510  0.518025  0.498566  12233.000000
weighted avg   0.544274  0.499223  0.502718  12233.000000

=== Final Results ===
Optimized Model AUC: 0.5544


In [79]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit

# Define core features that worked best
core_features = [
    # Strongest predictors from previous runs
    'skew_trend_10d', 'skew_volatility', 'log_moneyness_skew',
    'skew_trend_10d_squared', 'vol_adj_skew',
    
    # Volume-based features
    'log_option_volume', 'volume_spread',
    
    # Technical indicators
    'skew_ema_3d', 'skew_ema_10d', 'skew_rsi',
    
    # Momentum features
    'skew_momentum_5d', 'skew_ma_cross'
]

# Define ensemble models
models = {
    'GBM': GradientBoostingClassifier(
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
        min_samples_leaf=50,
        subsample=0.8,
        random_state=42
    ),
    'RF': RandomForestClassifier(
        n_estimators=200,
        max_depth=5,
        min_samples_leaf=50,
        max_features='sqrt',
        n_jobs=-1,
        random_state=42
    )
}

# Run experiments with ensemble models
for model_name, model in models.items():
    print(f"\n=== Testing {model_name} ===")
    results = run_experiment(model_data, core_features, model)
    
    # Print feature importances
    if hasattr(model, 'feature_importances_'):
        importances = pd.DataFrame({
            'feature': core_features,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\nTop 10 Feature Importances:")
        print(importances.head(10))


=== Testing GBM ===

Running experiment with 12 features
Shape after dropping NaNs: (73402, 68)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...

Average ROC AUC: 0.5551

Last Fold Results:
Train: 2024-08-02T00:00:00.000000000 to 2024-12-03T00:00:00.000000000
Val: 2024-12-03T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.632269  0.429674  0.511646   7515.000000
1              0.398541  0.601950  0.479568   4718.000000
accuracy       0.496117  0.496117  0.496117      0.496117
macro avg      0.515405  0.515812  0.495607  12233.000000
weighted avg   0.542125  0.496117  0.499274  12233.000000

Top 10 Feature Importances:
                   feature  importance
8             skew_ema_10d    0.349865
1          skew_volatility    0.125550
6            volume_spread    0.092585
5        log_option_volume    0.073199
11           skew_m

In [81]:
def create_focused_features(df):
    """Create enhanced feature set with infinity handling."""
    df = df.copy()
    
    # 1. EMA-based features
    df['skew_ema_3d'] = df.groupby('underlying_ticker')['log_moneyness_skew'].transform(
        lambda x: x.ewm(span=3, adjust=False).mean()
    )
    df['skew_ema_10d'] = df.groupby('underlying_ticker')['log_moneyness_skew'].transform(
        lambda x: x.ewm(span=10, adjust=False).mean()
    )
    
    # Safe division function
    def safe_divide(a, b, fill_value=0):
        result = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
        return np.clip(result, -1e6, 1e6)  # Clip extreme values
    
    # 2. Ratio and difference features (with safe operations)
    df['ema_ratio'] = safe_divide(df['skew_ema_3d'], df['skew_ema_10d'])
    df['ema_diff'] = df['skew_ema_3d'] - df['skew_ema_10d']
    
    # 3. Trend features (with safe operations)
    df['ema_trend'] = df.groupby('underlying_ticker')['skew_ema_10d'].transform(
        lambda x: x.pct_change(5).clip(-10, 10)  # Clip extreme changes
    )
    
    # 4. Volatility interactions
    df['vol_ema_ratio'] = df['skew_volatility'] * df['ema_ratio']
    df['vol_weighted_spread'] = df['skew_volatility'] * df['volume_spread']
    
    # 5. Volume-based signals (with clipping)
    df['volume_trend'] = df.groupby('underlying_ticker')['log_option_volume'].transform(
        lambda x: x.pct_change(5).clip(-10, 10)
    )
    df['spread_momentum'] = df.groupby('underlying_ticker')['volume_spread'].transform(
        lambda x: x.pct_change(3).clip(-10, 10)
    )
    
    return df

# Define focused feature set
focused_features = [
    # Core features
    'skew_ema_10d', 'skew_volatility', 'volume_spread',
    'skew_ema_3d', 'vol_adj_skew',
    
    # Derived features
    'ema_ratio', 'ema_diff', 'ema_trend',
    'vol_weighted_spread', 'volume_trend', 'spread_momentum'
]

# Create optimized GBM
focused_gbm = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=4,
    min_samples_leaf=30,
    subsample=0.85,
    random_state=42
)

# Run experiment
print("Creating focused features...")
model_data = create_focused_features(model_data)

print("\n=== Testing Focused GBM ===")
focused_results = run_experiment(model_data, focused_features, focused_gbm)

# Print feature importances
importances = pd.DataFrame({
    'feature': focused_features,
    'importance': focused_gbm.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importances:")
print(importances)

Creating focused features...

=== Testing Focused GBM ===

Running experiment with 11 features
Shape after dropping NaNs: (82147, 75)
Training fold 1/5...
Training fold 2/5...
Training fold 3/5...
Training fold 4/5...
Training fold 5/5...

Average ROC AUC: 0.5579

Last Fold Results:
Train: 2024-07-22T00:00:00.000000000 to 2024-11-29T00:00:00.000000000
Val: 2024-11-29T00:00:00.000000000 to 2024-12-24T00:00:00.000000000

Classification Report:
              precision    recall  f1-score       support
0              0.615313  0.405525  0.488863   8145.000000
1              0.418239  0.627660  0.501983   5546.000000
accuracy       0.495508  0.495508  0.495508      0.495508
macro avg      0.516776  0.516592  0.495423  13691.000000
weighted avg   0.535481  0.495508  0.494177  13691.000000

Feature Importances:
                feature  importance
0          skew_ema_10d    0.302049
1       skew_volatility    0.111252
9          volume_trend    0.100343
2         volume_spread    0.093511
10  

In [83]:
# Save the clean dataframe as CSV
output_path = '../data/processed/clean_df.csv'
clean_df.to_csv(output_path, index=False)

print(f"Dataset shape: {clean_df.shape}")
print(f"Saved to: {output_path}")

# Display first few rows and data info
print("\nFirst few rows:")
print(clean_df.head())
print("\nDataset info:")
print(clean_df.info())

OSError: Cannot save file into a non-existent directory: '../data/processed'

In [85]:
import os

# Create directories if they don't exist
os.makedirs('../data/processed', exist_ok=True)

# Save the clean dataframe as CSV
output_path = './data/processed/clean_df.csv'
clean_df.to_csv(output_path, index=False)

print(f"Dataset shape: {clean_df.shape}")
print(f"Saved to: {output_path}")

# Display first few rows and data info
print("\nFirst few rows:")
print(clean_df.head())
print("\nDataset info:")
print(clean_df.info())

Dataset shape: (125000, 33)
Saved to: ./data/processed/clean_df.csv

First few rows:
           date underlying_ticker       open       high        low      close  \
0    2024-07-01                AA  40.021413  40.637890  39.315446  39.474537   
1000 2024-07-02                AA  39.663458  40.916304  39.627663  40.598122   
2000 2024-07-03                AA  41.353798  42.755791  41.343857  42.139313   
3000 2024-07-05                AA  42.407781  42.855222  42.179084  42.198971   
4000 2024-07-08                AA  42.298404  42.308346  41.075390  41.423401   

         volume  total_option_volume  option_hhi  volume_weighted_hhi  ...  \
0     3228000.0              18243.0    0.044966             0.000033  ...   
1000  4039500.0              35060.0    0.172495             0.000252  ...   
2000  4074500.0              32198.0    0.071051             0.000104  ...   
3000  2670200.0              14792.0    0.033550             0.000016  ...   
4000  3001000.0              12452.0  